# 기출5



- 휴대폰 음질 측정 데이터로 음질의 특징을 분석하고자 한다. MOS(Mean Opinion Score) 변수는 두 지점간의 통신망과 테스트 장비를 이용하여 측정한 음질 측정 지표이며, 다음은 데이터셋에 대한 정보이다.

    phone.csv (구분자:  “,” , 105,828 Rows, 9 Columns)

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [2]:
import datetime
import locale

import scipy.stats as sc

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [3]:
import sklearn
sklearn.__version__

'0.21.3'

In [4]:
dat=pd.read_csv('test/phone.csv', na_values = ["null","NA","","n/a"])

dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105828 entries, 0 to 105827
Data columns (total 10 columns):
Date              105828 non-null object
Signal            105821 non-null float64
Speed             105828 non-null float64
Distance          95469 non-null float64
Manufacturer      10 non-null object
TestResult        105828 non-null object
TestTechnology    105828 non-null object
SetupTime         105828 non-null float64
MOS               105828 non-null int64
Acceptability     5050 non-null float64
dtypes: float64(5), int64(1), object(4)
memory usage: 8.1+ MB


In [5]:
dat.Result=dat.TestResult.astype('category')
dat.Technology=dat.TestTechnology.astype('category')
# dat.Xgrp=dat.Xgrp.astype('category')

* 다음의 전처리를 먼저 수행한 후 분석 진행하시오.

    단계 1-1. 데이터 중 테스트 결과(TestResult)가 SUCCESS이고 신호 강도(Signal) 값이 결측치(null, N/A 등)가 아닌 데이터만 추출





In [6]:
dat1=dat[(dat.TestResult=="SUCCESS")&(dat.Signal.notna())].copy()
dat1.shape
# [1] 105147      10


(105147, 10)

    단계 1-2. 결측치 보정 

    두 지점 간의 거리(Distance) 변수의 결측치는 통신망 유형별(TestTechnology)로 두 지점 간의 거리(Distance) 값이 결측치가 없는 데이터로 다음과 같은 선형식을 만들고 이를 이용하여 결측치를 보정(OLS(Ordinary Least Squares) 적용)

        Distan⁡ce=b_1×Signal



In [7]:
list2=['Distance', 'Signal']
b1_dat=dat1[dat1.Distance.notna()].copy()

for i in b1_dat.TestTechnology.unique() :
  b1_dat2=b1_dat[b1_dat.TestTechnology==i][list2]
  globals()['lm_'+str(i)]=LinearRegression(fit_intercept=False).fit(b1_dat2[['Signal']], b1_dat2[['Distance']])

In [8]:
dat2=dat1.copy()
dat2.Distance.isna().sum()

10185

In [9]:
for i in b1_dat.TestTechnology.unique() :
    dat2.loc[(dat2.TestTechnology==i) & (dat2.Distance.isna()), ['Distance']]=\
    eval('lm_'+str(i)).predict(dat2.loc[(dat2.TestTechnology==i) & (dat2.Distance.isna()), ['Signal']])

In [10]:
dat2.Distance.isna().sum()

0

In [11]:
len(dat2)

105147

통화 연결 시간(SetupTime)이 0인 데이터는 통신망 유형별(TestTechnology)로 ‘통화 연결 시간이 0이 아닌 데이터들의 평균 값’으로 보정



In [12]:
dat3=dat2.copy()
dat2.SetupTime=np.where(dat2.SetupTime == 0, np.nan, dat2.SetupTime)
dat3.SetupTime=np.where(dat3.SetupTime == 0, np.nan, dat3.SetupTime)

dat3['SetupTime']=dat3.groupby('TestTechnology')['SetupTime'].apply(lambda x: x.fillna(x.mean()))

In [13]:
dat3[dat2['SetupTime'].isna()].head()

,Date,Signal,Speed,Distance,Manufacturer,TestResult,TestTechnology,SetupTime,MOS,Acceptability
3,2017-07-01 0:08,-65.0,69.279999,1770.92,LGE,SUCCESS,UMTS,4.063578,1,0.0
5,2017-07-01 0:10,-61.0,68.860001,452.50,LGE,SUCCESS,UMTS,4.063578,1,0.0
7,2017-07-01 0:17,-73.0,70.019997,296.19,LGE,SUCCESS,UMTS,4.063578,1,0.0
14,2017-07-01 0:30,-83.0,31.030001,1385.85,NaN,SUCCESS,UMTS,4.063578,4,1.0
16,2017-07-01 0:32,-87.0,31.030001,17496.52,NaN,SUCCESS,UMTS,4.063578,1,0.0


전송 속도(Speed)가 0 이하인 데이터는 1) 단계와 동일하게 통신망 유형별(TestTechnology)로 전송 속도(Speed) 값이 0 초과(>)인 데이터를 사용하여 선형식을 만들고 이를 이용하여 추정해 보정

Speed=b_2×Signal

In [14]:
list3=['Speed', 'Signal']
b2_dat=dat3[dat3.Speed > 0].copy()

for i in b2_dat.TestTechnology.unique() :
  b2_dat2=b2_dat[b2_dat.TestTechnology==i][list3]
  globals()['lm_speed_'+str(i)]=LinearRegression(fit_intercept=False).fit(b2_dat2[['Signal']], b2_dat2[['Speed']])

In [15]:
dat4=dat3.copy()
(dat4.Speed <=0).sum()

58505

In [16]:
for i in b2_dat.TestTechnology.unique() :
    dat4.loc[(dat4.TestTechnology==i) & (dat4.Speed <= 0), ['Speed']]=\
    eval('lm_speed_'+str(i)).predict(dat4.loc[(dat4.TestTechnology==i) & (dat4.Speed <= 0), ['Signal']])

In [17]:
(dat4.Speed <=0).sum()

0

In [18]:
dat4.Speed[dat3.Speed <= 0].head()

13    14.590247
15    14.590247
17    14.590247
19    14.590247
20    14.590247
Name: Speed, dtype: float64

In [19]:
basetable1=dat4.copy()

In [20]:
len(basetable1)

105147

* 상기 전처리를 완료한 데이터 프레임(데이터 프레임명: basetable1)으로 다음 분석(문제 1~3) 진행하시오.

1. (basetable1 이용) 통신망 유형(TestTechnology)별로 이상치(Outlier)를 제거한 후 통신망 유형(TestTechnology)별로 통화 연결 시간(SetupTime)에 차이가 있는지 검정하고 검정 통계량(F Statistic)을 기술하시오.(이때 이상치는 Tukey 방식 적용하며, 검정 통계량은 정수부분만 기술)

    - [Tukey 방식] 
        Q1: 제 1사분위수, Q3: 제 3사분위수, IQR: Q3 – Q1 일 때, 
        (Y < Q1 - 1.5*IQR) or (Y > Q3 + 1.5*IQR) 이면 Y 이상치
        정답	891661.2777499713

In [21]:
def outlier_test(x):
    Q1=x.quantile(1/4)
    Q3=x.quantile(3/4)
    IQR=Q3-Q1
    LL=Q1-(1.5*IQR)
    UU=Q3+(1.5*IQR)
    outlier_out= (x < LL) | (x > UU)
    return outlier_out

In [22]:
outlier_result=basetable1.groupby('TestTechnology')['SetupTime'].apply(outlier_test)

In [23]:
Q1 = basetable1[~outlier_result]

In [24]:
Q1_out=ols('SetupTime~TestTechnology', data=Q1).fit()
Q1_out2=anova_lm(Q1_out)
print(Q1_out2)

                     df         sum_sq        mean_sq              F  PR(>F)
TestTechnology      2.0  229000.414216  114500.207108  891661.277751     0.0
Residual        97045.0   12461.764210       0.128412            NaN     NaN


In [25]:
Q1_ans=Q1_out2['F'][0]

print('Q1 정답: ', Q1_ans)  #  891661.2777504099

Q1 정답:  891661.2777507299


단계 1-3. (basetable1 이용) 테스트 시점의 날짜(Date)의 요일 변수(Day) 생성 후 요일 변수(Day)와 통신망 유형(TestTechnology) 변수의 가변수(Dummy)를 생성하시오.


In [26]:
basetable1.Date.dtype

dtype('O')

In [27]:
basetable1.Date.head()

0    2017-07-01 0:00
1    2017-07-01 0:02
2    2017-07-01 0:05
3    2017-07-01 0:08
4    2017-07-01 0:10
Name: Date, dtype: object

In [28]:
Q2=basetable1.copy()

In [29]:
Q2.Date=pd.to_datetime(Q2.Date)

In [30]:
import locale
locale.setlocale(locale.LC_ALL,'')

'Korean_Korea.949'

In [31]:
# Q2['Day']=Q2.Date.dt.day_name('ko_kr')
Q2['Day']=Q2.Date.dt.day_name('Korean_Korea.949')
Q2['Day'].head()

0    토요일
1    토요일
2    토요일
3    토요일
4    토요일
Name: Day, dtype: object

In [32]:
# pd.get_dummies?

In [33]:
Q2_2=pd.get_dummies(Q2, columns=['TestTechnology', 'Day'], drop_first=True)
Q2_2.head(3)
Q2_2['TestTechnology']=Q2.TestTechnology

단계 1-4. (단계1-3. 이용) Split Data 함수를 사용하여 Train과 Test 데이터를 분할하시오.  

    - Train:Test = 7:3 비율, 음질의 만족 여부(Acceptability)에 따라 층화추출,  seed : 123


In [34]:
Q2_2.Acceptability.head()

0    0.0
1    1.0
2    0.0
3    0.0
4    1.0
Name: Acceptability, dtype: float64

In [35]:
Q2_2.Acceptability.isna().sum()

100149

In [36]:
Q2_3=Q2_2.dropna(subset=['Acceptability'])

In [37]:
Q2_3.Acceptability.value_counts()

1.0    3790
0.0    1208
Name: Acceptability, dtype: int64

In [38]:
Q2_2.Acceptability=Q2_2.Acceptability

In [39]:
train, test=train_test_split(Q2_3, test_size=0.3, 
                 random_state=123, 
                 stratify=Q2_3.Acceptability)

In [40]:
len(train)

3498

In [41]:
len(test)

1500

2. (단계 1-3. 생성된 데이터셋 이용) 음질의 만족 여부(Acceptability)를 예측하는 KNN 분류 모델을 만드시오.
 - Train Data 사용
 - 독립변수(총 6개): 신호 강도(Signal), 전송 속도(Speed), 두 지점 간의 거리(Distance), 통화 연결 시간(SetupTime), 통신망 유형(TestTechnology)의 가변수(2개)
생성된 모델에 Test 데이터를 적용하여 음질의 만족 여부(Acceptability)를 예측한 후 모델의 성능평가로 AUC(the Area Under a ROC Curve) 값을 기술하시오.
- KNN 분류 모델 가이드 : K=5
- AUC 값은 소수점 셋째 자리 이하는 버리고 둘째 자리까지 기술
정답	0.93

In [42]:
train.columns

Index(['Date', 'Signal', 'Speed', 'Distance', 'Manufacturer', 'TestResult',
       'SetupTime', 'MOS', 'Acceptability', 'TestTechnology_LTE',
       'TestTechnology_UMTS', 'Day_목요일', 'Day_수요일', 'Day_월요일', 'Day_일요일',
       'Day_토요일', 'Day_화요일', 'TestTechnology'],
      dtype='object')

In [43]:
x_var1=['Signal','Speed','Distance', 'SetupTime','TestTechnology_LTE','TestTechnology_UMTS' ]
Q2_KNN=KNeighborsClassifier(5).fit(train[x_var1], train['Acceptability'])

Q2_pred=Q2_KNN.predict_proba(test[x_var1])

In [44]:
Q2_pred

array([[1. , 0. ],
       [0.4, 0.6],
       [0.8, 0.2],
       ...,
       [0. , 1. ],
       [0. , 1. ],
       [0. , 1. ]])

In [45]:
Q2_ans=roc_auc_score(test['Acceptability'], Q2_pred[:,1])
print('Q2 정답: ', Q2_ans)  # 0.9126828370052165

Q2 정답:  0.9126828370052165


3. 음질의 만족 여부(Acceptability ,Target)를 예측하는 모델을 Decision Tree 알고리즘을 이용하여 만들고자 한다. 통신망 유형(TestTechnology)별로 분리해서 모델을 다음 조건에 따라 만드시오.
 - Train Data 사용
- 독립변수(총 10개): 신호 강도(Signal), 전송 속도(Speed), 두 지점 간의 거리(Distance), 통화 연결 시간(SetupTime), 요일의 가변수(6개)
 
이 모델에 Test데이터를 적용하여 음질의 만족 여부(Target)를 예측한 후 정확도(Accuracy) 값을 기술하시오.
- [Decision Tree 모델 가이드] 
분순도 기준: Gini, Max Depth: 7, Min Samples Splits: 5, Seed: 1234, 그 외: Default
- 정확도(Accuracy) 값은 소수점 셋째 자리까지 기술, 이후 자리는 절삭.
정답	0.904

In [46]:
x_var2=['Signal','Speed','Distance', 'SetupTime', 
        'Day_목요일', 'Day_수요일', 'Day_월요일', 'Day_일요일','Day_토요일', 'Day_화요일']

In [47]:
train.TestTechnology.unique()

array(['LTE', 'UMTS', 'GSM'], dtype=object)

In [48]:
train.columns

Index(['Date', 'Signal', 'Speed', 'Distance', 'Manufacturer', 'TestResult',
       'SetupTime', 'MOS', 'Acceptability', 'TestTechnology_LTE',
       'TestTechnology_UMTS', 'Day_목요일', 'Day_수요일', 'Day_월요일', 'Day_일요일',
       'Day_토요일', 'Day_화요일', 'TestTechnology'],
      dtype='object')

In [49]:
for i in train.TestTechnology.unique():
    temp=train[train.TestTechnology==i]
    globals()['dt_'+str(i)]=DecisionTreeClassifier(max_depth=6,
                           min_samples_split=5, 
                           random_state=1234).fit(temp[x_var2], temp['Acceptability'])

In [50]:
pred=[]
target=[]
for i in test.TestTechnology.unique():
    temp=test[test.TestTechnology==i]
    pred1=eval('dt_'+str(i)).predict(temp[x_var2])
    # target.append(temp['Acceptability'])
    target=target+list(temp['Acceptability'])
    pred=pred+list(pred1)
    

In [51]:
target

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0

In [52]:
from sklearn.metrics import confusion_matrix

confusion_matrix(target, pred)

array([[ 330,   33],
       [ 109, 1028]], dtype=int64)

In [53]:
Q3_ans=accuracy_score(target, pred)
print('Q3 정답: ', Q3_ans)  # 0.898

Q3 정답:  0.9053333333333333
